<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alternating Least Squares

This notebook will ALS Matrix Factorization algorithm to recommend and rank Top10 items based on the paper of Koren, Bell and Volinsky, 2009 (https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf)

The used library:

*   Implicit ALS(https://benfred.github.io/implicit/ by Ben Frederickson)

In [1]:
!pip install --upgrade implicit

In [2]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta

# Loading DataFrames



In [3]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-3-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [4]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


In [5]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2220299 17697 77401


# Data Preprocessing

In [6]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)
# Entry Date to date
df['Entry Date'] = pd.to_datetime(df['Entry Date'], format='%d.%m.%y')
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"] > 0]
# dropping backlog invoices, Specified date to filter the rows
specific_date = pd.to_datetime('2022-11-01')
# Filter the DataFrame to keep only the rows that are before or equal to the specific date
df = df[df["Entry Date"] >= specific_date]
#insert purchase indication column
df["purchase"] = 1

In [7]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2028956 13894 75643


In [8]:
# add column that represent sku count
def sku_count(df_pl):

  df_pl['sku_count'] = df_pl.groupby('BranchCustomerNbr')['Sku'].transform('nunique')

  return df_pl

In [9]:
df = sku_count(df)

In [10]:
# drop customers that only purchased 1 SKU
#df = df[df["sku_count"] > 1]

In [11]:
print(len(df), len(df["BranchCustomerNbr"].unique()), len(df["Sku"].unique()))

2028956 13894 75643


In [12]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [13]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,...,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase,sku_count,bcn_id,sku_id
213,1545306,2022-11-02,NaN,2022FM11,15885514,AXIS SOLUTION (PRIVATE) LIMITED,Export Channel (DE),,CF55877,DT PRINT ZQ320 KIT LABEL SENSOR,...,5805,AIDC/PoS Printers,Mobile Receipt Printer,"Other (incl. AIDC/POS, V7)","10.713,30",41.0,1,20,497,49835
458,4422886,2022-11-03,NaN,2023FM02,44413224,BWG INFORMATIONSYSTEME GMBH,Business Channel,,CF89211,Z-SELECT 2000D REMOVABLE NS,...,5812,AIDC/PoS Printers,Label Printers Supplies,"Other (incl. AIDC/POS, V7)","1.393,00",140.0,1,111,1000,52759


In [14]:
users = df.bcn_id.unique()
skus = df.sku_id.unique()
print(len(users), len(skus))

13894 75643


In [15]:
grouped_df = df.groupby(["bcn_id", "sku_id"]).agg({
          "Qty Shipped":"sum",
          "purchase":"sum"}).reset_index()

In [16]:
# create binary column
grouped_df["purch_bin"] = 1

In [17]:
grouped_df.head(2)

,bcn_id,sku_id,Qty Shipped,purchase,purch_bin
0,0,928,1.0,1,1
1,0,3422,1.0,1,1


In [18]:
grouped_df_2 = grouped_df.copy()

## Negative Sampling

In [19]:
def negative_sampling(df_pl, bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param sku_ids: list of sku ids
    @param items: unique list of sku ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'purchase'])

    #df_train = df_train[['bcn_id', 'sku_id']].assign(purchase=1)
    df_pl = pd.concat([df_pl, df_neg], ignore_index=True)

    return df_pl

In [20]:
neg_grouped_df = negative_sampling(grouped_df_2, users, skus, skus, n_neg=100)

In [21]:
neg_grouped_df = neg_grouped_df.fillna(0)

In [22]:
print(len(grouped_df), len(neg_grouped_df))

544417 1933817


## ALS Model param tuning

In [23]:
import implicit
from implicit.gpu.als import AlternatingLeastSquares as ALS
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k, train_test_split

In [24]:
csr_qty_matrix = sparse.csr_matrix((grouped_df['Qty Shipped'], (grouped_df['bcn_id'], grouped_df['sku_id'])))
csr_freq_matrix = sparse.csr_matrix((grouped_df['purchase'], (grouped_df['bcn_id'], grouped_df['sku_id'])))
csr_bin_matrix = sparse.csr_matrix((grouped_df['purch_bin'], (grouped_df['bcn_id'], grouped_df['sku_id'])))

In [25]:
neg_csr_qty_matrix = sparse.csr_matrix((neg_grouped_df['Qty Shipped'], (neg_grouped_df['bcn_id'], neg_grouped_df['sku_id'])))
neg_csr_freq_matrix = sparse.csr_matrix((neg_grouped_df['purchase'], (neg_grouped_df['bcn_id'], neg_grouped_df['sku_id'])))
neg_csr_bin_matrix = sparse.csr_matrix((neg_grouped_df['purch_bin'], (neg_grouped_df['bcn_id'], neg_grouped_df['sku_id'])))

In [26]:
csr_qty_matrix_train, csr_qty_matrix_test = leave_k_out_split(csr_qty_matrix, K=1, random_state=None)
csr_freq_matrix_train, csr_freq_matrix_test = leave_k_out_split(csr_freq_matrix, K=1, random_state=None)
csr_bin_matrix_train, csr_bin_matrix_test = leave_k_out_split(csr_bin_matrix, K=1, random_state=None)

In [27]:
csr_qty_matrix_train_2, csr_qty_matrix_test_2 = train_test_split(csr_qty_matrix, train_percentage=0.9, random_state=None)
csr_freq_matrix_train_2, csr_freq_matrix_test_2 = train_test_split(csr_freq_matrix, train_percentage=0.9,  random_state=None)
csr_bin_matrix_train_2, csr_bin_matrix_test_2 = train_test_split(csr_bin_matrix, train_percentage=0.9, random_state=None)

In [28]:
neg_csr_qty_matrix_train, neg_csr_qty_matrix_test = leave_k_out_split(neg_csr_qty_matrix, K=1, random_state=None)
neg_csr_freq_matrix_train, neg_csr_freq_matrix_test = leave_k_out_split(neg_csr_freq_matrix, K=1, random_state=None)
neg_csr_bin_matrix_train, neg_csr_bin_matrix_test = leave_k_out_split(neg_csr_bin_matrix, K=1, random_state=None)

In [29]:
model = ALS()

In [30]:
model.fit(csr_freq_matrix_train_2)

  0%|          | 0/15 [00:00<?, ?it/s]

In [31]:
auc10 = AUC_at_k(model, csr_freq_matrix_train_2, csr_freq_matrix_test_2, K=10, num_threads=10)
auc10

  0%|          | 0/6850 [00:00<?, ?it/s]

0.5549555055348879

## Hyperparametertuning


In [32]:
from IPython.utils.sysinfo import num_cpus
from sklearn.model_selection import ParameterGrid

# Grid of hyperparameters to search
param_grid = {
    'factors': [100, 150, 200],
    'iterations':[10, 15, 20],
    'alpha': [5, 10, 20, 40],
    'regularization': [0.01, 0.1]
}

best_auc = -np.inf
best_params = {}

# Iterate through all parameter combinations
for params in ParameterGrid(param_grid):
    model = ALS(factors=params['factors'],
                    iterations=params['iterations'],
                    alpha=params['alpha'],
                    regularization=params['regularization'])
    model.fit(csr_qty_matrix_train, show_progress=False)

    auc10 = AUC_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test,
                    K=10, num_threads=10, show_progress=False)

    if auc10 > best_auc:
        best_auc = auc10
        best_params = params

# Print the best parameters and AUC
print("Best parameters:", best_params)
print("Best AUC:", best_auc)

Best parameters: {'alpha': 5, 'factors': 200, 'iterations': 20, 'regularization': 0.01}
Best AUC: 0.5870360498446862


In [33]:
best_auc_2 = -np.inf
best_params_2 = {}

# Iterate through all parameter combinations
for params in ParameterGrid(param_grid):
    model_2 = ALS(factors=params['factors'],
                    iterations=params['iterations'],
                    alpha=params['alpha'],
                    regularization=params['regularization'])
    model_2.fit(csr_freq_matrix_train, show_progress=False)

    auc10_2 = AUC_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test,
                    K=10, num_threads=10, show_progress=False)

    if auc10_2 > best_auc_2:
        best_auc_2 = auc10_2
        best_params_2 = params

# Print the best parameters and AUC
print("Best parameters:", best_params_2)
print("Best AUC:", best_auc_2)

Best parameters: {'alpha': 10, 'factors': 200, 'iterations': 10, 'regularization': 0.1}
Best AUC: 0.5909200946060084


In [34]:
model = ALS(**best_params)
model.fit(csr_qty_matrix_train)
auc10 = AUC_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)
prec10 = precision_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)
map10 = mean_average_precision_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)
ndcg10 = ndcg_at_k(model, csr_qty_matrix_train, csr_qty_matrix_test, K=10, num_threads=10)

print(f"AUC@10: {auc10}; Prec@10: {prec10}; Map@10: {map10}; Ndcg@10: {ndcg10}")

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

AUC@10: 0.586259374517987; Prec@10: 0.17263877381938691; Map@10: 0.09440059474494034; Ndcg@10: 0.11290784791177139


In [35]:
model_2 = ALS(**best_params_2)
model_2.fit(csr_freq_matrix_train)
auc10 = AUC_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)
prec10 = precision_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)
map10 = mean_average_precision_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)
ndcg10 = ndcg_at_k(model_2, csr_freq_matrix_train, csr_freq_matrix_test, K=10, num_threads=10)

print(f"AUC@10: {auc10}; Prec@10: {prec10}; Map@10: {map10}; Ndcg@10: {ndcg10}")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

  0%|          | 0/9656 [00:00<?, ?it/s]

AUC@10: 0.5904024187585416; Prec@10: 0.180923777961889; Map@10: 0.10263299568653773; Ndcg@10: 0.12113538633531427


## Recommending with ALS

In [36]:
# add column that represent sku count
def sku_count_2(df_pl):

  df_pl['sku_count'] = df_pl.groupby('bcn_id')['sku_id'].transform('nunique')

  return df_pl

In [37]:
grouped_df

,bcn_id,sku_id,Qty Shipped,purchase,purch_bin
0,0,928,1.0,1,1
1,0,3422,1.0,1,1
2,0,3423,4.0,2,1
3,0,3424,3.0,1,1
4,0,4189,4.0,1,1
...,...,...,...,...,...
544412,13891,42310,2.0,1,1
544413,13891,45247,2.0,1,1
544414,13892,46407,2.0,1,1
544415,13893,36049,25.0,1,1


In [38]:
df_2 = sku_count_2(grouped_df)

In [39]:
users_less_20 = df_2[(df_2.sku_count <= 20) & (df_2.sku_count >= 5) ]["bcn_id"].unique()
users_more_20 = df_2[df_2.sku_count > 20]["bcn_id"].unique()

In [40]:
sku_list = df[["sku_id", "Product Descr1", "ProductGroupDescription", "ProductGroupMasterDescription"]].drop_duplicates()
sku_list.head(1)

,sku_id,Product Descr1,ProductGroupDescription,ProductGroupMasterDescription
213,49835,DT PRINT ZQ320 KIT LABEL SENSOR,Mobile Receipt Printer,AIDC/PoS Printers


In [41]:
from google.colab import files

In [42]:
 # Assuming you want recommendations for user with ID 10
import random
user_id = random.choice(users_less_20)
user_id_2 = random.choice(users_more_20)
print(user_id)
print(user_id_2)

10932
3966


In [43]:
user_purchases = pd.DataFrame(df_2[(df.bcn_id == user_id)])
user_purchases = user_purchases.groupby(["bcn_id","sku_id"]).agg({
                                        "Qty Shipped":"sum",
                                        "purchase":"sum"}).reset_index()
user_purchases = user_purchases.sort_values(by="Qty Shipped", ascending=False)
user_purchases = user_purchases.merge(sku_list, on="sku_id", how="left")
user_purchases.to_csv(f'{user_id}_purchase_history.csv')
files.download(f'{user_id}_purchase_history.csv')
user_purchases

<ipython-input-43-f8c97ad2b10a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  user_purchases = pd.DataFrame(df_2[(df.bcn_id == user_id)])


IndexingError: ignored

In [ ]:
user_purchases_2 = pd.DataFrame(df_2[(df.bcn_id == user_id_2)])
user_purchases_2 = user_purchases_2.groupby(["bcn_id","sku_id"]).agg({
                                        "Qty Shipped":"sum",
                                        "purchase":"sum"}).reset_index()
user_purchases_2 = user_purchases_2.sort_values(by="Qty Shipped", ascending=False)
user_purchases_2 = user_purchases_2.merge(sku_list, on="sku_id", how="left")
user_purchases_2.to_csv(f'{user_id_2}_purchase_history.csv')
files.download(f'{user_id_2}_purchase_history.csv')
len(user_purchases_2)

## Recommendations

#### Model 1

In [ ]:
# Now you can call the recommend function
userid = [user_id]
ids, scores = model.recommend(userid, csr_qty_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id}_ALS_QTY_REC.csv')
files.download(f'{user_id}_ALS_QTY_REC.csv')
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

In [ ]:
# Now you can call the recommend function
userid = [user_id_2]
ids, scores = model.recommend(userid, csr_qty_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id_2}_ALS_QTY_REC.csv')
files.download(f'{user_id_2}_ALS_QTY_REC.csv')
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

#### Model 2

In [ ]:
# Now you can call the recommend function
userid = [user_id]
ids, scores = model_2.recommend(userid, csr_freq_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id}_FREQ_ALS_REC.csv')
files.download(f'{user_id}_FREQ_ALS_REC.csv')
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

In [ ]:
# Now you can call the recommend function
userid = [user_id_2]
ids, scores = model_2.recommend(userid, csr_freq_matrix[userid], N=10, filter_already_liked_items=True)
ids, scores

rec_tab = pd.DataFrame(data=[ids[0],scores[0]])
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"score", 2:"sku_id", 3:"score"})
rec_tab = rec_tab.merge(sku_list, on="sku_id", how="left")
rec_tab.to_csv(f'{user_id_2}_FREQ_ALS_REC.csv')
files.download(f'{user_id_2}_FREQ_ALS_REC.csv')
rec_tab
# rec_tab.sort_values(by="conf", ascending=False)

# Test LibRecommender

In [44]:
!pip install LibRecommender

In [45]:
import os
import datetime
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [46]:
from libreco.data import split_by_num, random_split
from libreco.data import DatasetFeat

Instructions for updating:
non-resource variables are not supported in the long term


In [47]:
df_4 = df_2.copy()

In [ ]:
df_2 = df_2.rename(columns={"bcn_id":"user", "sku_id":"item", "purchase":"label"})

In [54]:
df_4 = df_4.rename(columns={"bcn_id":"user", "sku_id":"item", "purch_bin":"label"})

In [56]:
train, test, eval = random_split(df_4[["user", "item", "label"]], multi_ratios=[0.8,0.1,0.1])

In [57]:
train, data_info = DatasetFeat.build_trainset(train)
eval = DatasetFeat.build_evalset(eval)
test = DatasetFeat.build_testset(test)

In [58]:
train_2, eval_2 = split_by_num(df_4[["user", "item", "label"]], test_size=1)

In [59]:
train_2, data_info_2 = DatasetFeat.build_trainset(train_2)
eval_2 = DatasetFeat.build_evalset(eval_2)

In [60]:
from libreco.algorithms import ALS as ALS_lib
from libreco.evaluation import evaluate
from libreco.data import random_split, DatasetPure

In [61]:
metrics = [
        "loss",
        "balanced_accuracy",
        "roc_auc",
        "precision",
        "recall",
        "map",
        "ndcg",
    ]

In [62]:
model = ALS_lib(
    "ranking",
    data_info=data_info,
    embed_size=16,
    n_epochs=10,
    reg=0.01,
    alpha=20,
    n_threads=10,
)

In [63]:
model.fit(train_data=train,
          neg_sampling=True,
          verbose=1,
          shuffle=False,
          eval_data=eval,
          metrics=metrics,
          n_neg=100
          )

Training start time: 2023-08-10 20:42:57
Epoch 1 elapsed: 1.043s
Epoch 2 elapsed: 1.085s
Epoch 3 elapsed: 1.295s
Epoch 4 elapsed: 1.269s
Epoch 5 elapsed: 1.249s
Epoch 6 elapsed: 1.050s
Epoch 7 elapsed: 0.983s
Epoch 8 elapsed: 1.479s
Epoch 9 elapsed: 0.985s
Epoch 10 elapsed: 1.058s


In [64]:
eval_result = evaluate(model=model,
        data=eval,
        neg_sampling=True,
        eval_batch_size=2048,
        k=10,
        metrics=metrics)
eval_result

random neg item sampling elapsed: 0.054s


eval_listwise: 100%|██████████| 6396/6396 [00:17<00:00, 373.22it/s]


{'loss': 0.5952163372634862,
 'balanced_accuracy': 0.6987252378957448,
 'roc_auc': 0.9014089149821349,
 'precision': 0.05395559724828018,
 'recall': 0.08717600643236019,
 'map': 0.11410567875464608,
 'ndcg': 0.14740101897690855}